In [2]:
#----------------------------------------------------------------------------------------
#Imports
#========================================================================================

#Import entire cmath library
#from cmath import *

#Import entire pandas library as pd
#import pandas as pd

#This is used for variable display in the GUI
#import sys

#Import entire numpy library
from numpy import *

#Import scipy.special library
from scipy.special import *

#Import entire matplotlib.pyplot library as plt
from matplotlib.pyplot import *

In [3]:
#----------------------------------------------------------------------------------------
# Inputs
#========================================================================================
#nu1: Longitudinal wave speed
v1 = 1100 #m/s

#Xi1: Longitudinal wave attenuation
X1 =  2.4 #dB/MHz/m

#nu2: Shear wave speed
v2 = 570 #m/s

#Xi2: Shear wave attenuation
X2 = X1*v1/v2 #dB/Mhz/m

#rho: Desity of the Medium
p = 1030 #kg/m^3

#rho2: Density of the crystal
p2 = 1910 #kg/m^3

#Phi': Amplitude of the Incident Wave
P1 = 0.000001 #m

#a: radius of the chrystal
a = 0.00025 #m

#f: frequency
f = 500000 #Hz

#k: Thermal Conductivity
k1 = 0.27 #W/(m-K)

#gamma: Thermal Diffusivity
gamma = 1.02*10**-7 #m^2/s

#U0: Thermal Diffusivity
h = 5 #W/(m^2-K)

In [4]:
#----------------------------------------------------------------------------------------
# Parameter Equations
#========================================================================================

#Cotangent function
def cot(tha):
    if tha == 0 or tha == pi:
        return 0
    else:
        return 1/tan(tha)

#Ratio of densities
eta = p/p2

#Tangents
tan1 = X1*v1*100/(pi*8.686*10**6)
tan2 = X2*v2*100/(pi*8.686*10**6)

#mus
u_e = v2**2*p
u_v = u_e*tan2
u1 = u_e + u_v* 1j

#omega: angular frequency
w = 2*pi*f

#lambdas
x_e = v1**2*p-2*u_e
x_v = x_e * tan1
x = x_e + x_v * 1j

#
E1 = (u1*(3*x + 2*u1))/(x + u1)

#
nu1 = x/(2*(x + u1))

#alpha_1 = Complex Longitudinal Wave Number
a1 = sqrt(w**2*p/(x + 2 * u1))

#Betta_1: Complex Shear Wave Number
B1 = sqrt(p*w**2/u1)

#Phi0: Wave Potential Amplitude
P = P1*exp(1j*a1*a*20)/a1

In [1]:
#----------------------------------------------------------------------------------------
#Imported scipy funcations
#========================================================================================

#P_n: Legendre Polynomials
def Pn(n):
    return legendre(n)
#n: int
#Degree of the polynomial.

#d(P_n)/dtheta: derivatives of Legendre Polynomials
def dPndtha(m,n,x,tha):
    if n == 0:
        return 0
        
    elif n >= 1:
        if m == 1:
            return -lpmv(1,n,x)
        elif m == 2:
            return -lpmv(1,n,x)*(cot(tha))
        else:
            raise TypeError("Invlaid input. The first term should be a 1 or a 2")
        
    elif n > 1:
        if m == 1:
            return -lpmv(1,n,x)
        elif m == 2:
            return lpmv(2,n,x) - lpmv(1,n,x)*(1/tan(tha))
        else:
            raise TypeError("Invlaid input. The first term should be a 1 or a 2")

#h_n: Spherical Hankel Function of the first kind
def hn(n,z):
    return sqrt( pi/(2*z) ) * hankel1(n+0.5,z)
#n: array_like
#Order (float)

#z: array_like
#Argument (float or complex)
#Basically whatever is in the parentheses


#j_n: Spherical Bessel Function of the first kind
def jn(n,z):
    return spherical_jn(n,z)
#n: int, array_like
#Order of the Bessel function (n >= 0)

#z: complex or float, array_like
#Argument of the Bessel function
#Basically whatever is in the parentheses

In [6]:
#----------------------------------------------------------------------------------------
#
#========================================================================================

#A_n
def An(n):
    if n == 1:
        e11 = (1-eta)*hn(1,a1*a) - a1*a*hn(2,a1*a)
        e21 = (1-eta)*hn(1,a1*a)
        e12 = -2*(1-eta)*hn(1,B1*a)
        e22 = -2*(1-eta)*hn(1,B1*a) + B1*a*hn(2,B1*a)
        e1  = -3*1j*((1-eta)*jn(1,a1*a) - a1*a*jn(2,a1*a))
        e2  = -3*1j*(1-eta)*jn(1,a1*a)
        
        mat = array([[e11,e12],[e21,e22]])
        av = P*array([[e1],[e2]])
        x = linalg.solve(mat,av)
    
        An = x[0][0]
        Bn = x[1][0]
    
    else:
        E11 = n*hn(n,a1*a) - a1*a*hn(n+1,a1*a)
        E12 = -n*(n+1)*hn(n,B1*a)
        E21 = hn(n,a1*a)
        E22 = -(n+1)*hn(n,B1*a) + B1*a*hn(n+1,B1*a)
        E1  = -1j**n*(2*n+1)*(n*jn(n,a1*a) - a1*a*jn(n+1,a1*a))
        E2  = -1j**n*(2*n+1)*jn(n,a1*a)
    
        mat = array([[E11,E12],[E21,E22]])
        av = P*array([[E1],[E2]])
        x = linalg.solve(mat,av)
    
        An = x[0][0]
        Bn = x[1][0]
    return An

#B_n
def Bn(n):
    if n == 1:
        e11 = (1-eta)*hn(1,a1*a) - a1*a*hn(2,a1*a)
        e21 = (1-eta)*hn(1,a1*a)
        e12 = -2*(1-eta)*hn(1,B1*a)
        e22 = -2*(1-eta)*hn(1,B1*a) + B1*a*hn(2,B1*a)
        e1  = -3*1j*((1-eta)*jn(1,a1*a) - a1*a*jn(2,a1*a))
        e2  = -3*1j*(1-eta)*jn(1,a1*a)
    
        mat = array([[e11,e12],[e21,e22]])
        av = P*array([[e1],[e2]])
        x = linalg.solve(mat,av)
    
        An = x[0][0]
        Bn = x[1][0]
    
    else:
        E11 = n*hn(n,a1*a) - a1*a*hn(n+1,a1*a)
        E12 = -n*(n+1)*hn(n,B1*a)
        E21 = hn(n,a1*a)
        E22 = -(n+1)*hn(n,B1*a) + B1*a*hn(n+1,B1*a)
        E1  = -1j**n*(2*n+1)*(n*jn(n,a1*a) - a1*a*jn(n+1,a1*a))
        E2  = -1j**n*(2*n+1)*jn(n,a1*a)
    
        mat = array([[E11,E12],[E21,E22]])
        av = P*array([[E1],[E2]])
        x = linalg.solve(mat,av)
    
        An = x[0][0]
        Bn = x[1][0]
    return Bn